In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["WEATHER_API_KEY"] = os.getenv("WEATHER_API_KEY")
WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")

In [3]:
import requests

async def get_weather_from_weatherapi(city: str) -> str:
    try:
        url = f"http://api.weatherapi.com/v1/forecast.json?key={WEATHER_API_KEY}&q={city}&days=3&aqi=no&alerts=no"
        response = requests.get(url)
        
        # Ensure valid JSON
        if response.status_code != 200:
            return f"❗ API request failed with status code {response.status_code}"
        
        data = response.json()

        if "error" in data:
            return f"❗ API Error: {data['error'].get('message', 'Unknown error')}"

        location = data["location"]
        current = data["current"]
        forecast = data["forecast"]["forecastday"]

        report = (
            f"📍 {location['name']}, {location['country']}\n"
            f"🌡️ Temp: {current['temp_c']}°C (Feels like {current['feelslike_c']}°C)\n"
            f"💧 Humidity: {current['humidity']}% | ☁️ Cloud: {current['cloud']}%\n"
            f"🌬️ Wind: {current['wind_kph']} kph\n"
            f"🌤️ Condition: {current['condition']['text']}\n\n"
            f"🗓️ Forecast:\n"
        )

        for day in forecast:
            date = day["date"]
            day_data = day["day"]
            report += (
                f"{date}: 🌡️ {day_data['avgtemp_c']}°C, 🌧️ {day_data['condition']['text']}, "
                f"💧 Humidity: {day_data['avghumidity']}%, 🌬️ Wind: {day_data['maxwind_kph']} kph\n"
            )

        return report

    except Exception as e:
        return f"❗ Error: {str(e)}"

In [4]:
import nest_asyncio
nest_asyncio.apply()

if __name__ == "__main__":
    import asyncio
    city = "Ongole"
    result = asyncio.run(get_weather_from_weatherapi(city))
    print("\n✅ Weather Report:\n")
    print(result)


✅ Weather Report:

📍 Ongole, India
🌡️ Temp: 35.1°C (Feels like 37.9°C)
💧 Humidity: 39% | ☁️ Cloud: 32%
🌬️ Wind: 26.3 kph
🌤️ Condition: Partly Cloudy

🗓️ Forecast:
2025-06-17: 🌡️ 32.3°C, 🌧️ Partly Cloudy , 💧 Humidity: 49%, 🌬️ Wind: 32.4 kph
2025-06-18: 🌡️ 33.6°C, 🌧️ Sunny, 💧 Humidity: 44%, 🌬️ Wind: 32.4 kph
2025-06-19: 🌡️ 33.2°C, 🌧️ Patchy rain nearby, 💧 Humidity: 46%, 🌬️ Wind: 34.2 kph

